<a href="https://colab.research.google.com/github/SasukeUchiha-05/NLP/blob/main/NLP_P7_22BD1A6612_MVS_Karthik_15_04_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement:
### 7.	Develop a Named Entity Recognition system: Use a dataset with annotated entities (e.g., persons, organizations).
- i.	Implement an NER model.
- ii.	Evaluate the model's performance.

### Step 1: Import Libraries and datasets.

In [ ]:
import spacy
import random
from spacy.training import Example
from datasets import load_dataset
from sklearn.metrics import classification_report
from spacy.tokens import DocBin
from spacy.training.iob_utils import offsets_to_biluo_tags

In [ ]:
conll = load_dataset("conll2003")

ner_label_map = conll["train"].features["ner_tags"].feature.int2str

Using the latest cached version of the module from C:\Users\tanay\.cache\huggingface\modules\datasets_modules\datasets\conll2003\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98 (last modified on Wed Apr  2 11:44:32 2025) since it couldn't be found locally at conll2003, or remotely on the Hugging Face Hub.


### Step 2: Conll Data to SpaCy Format. Preparing Training and Testing Data.



In [ ]:
def convert_conll_to_spacy(conll_dataset):
    spacy_data = []

    for entry in conll_dataset:
        text = " ".join(entry["tokens"])
        entities = []
        start = 0

        for token, label in zip(entry["tokens"], entry["ner_tags"]):
            token_start = text.find(token, start)
            token_end = token_start + len(token)

            if label != 0:
                entity_label = ner_label_map(label)
                entities.append((token_start, token_end, entity_label))

            start = token_end

        spacy_data.append((text, {"entities": entities}))

    return spacy_data

In [ ]:
train_data_spacy = convert_conll_to_spacy(conll["train"])
test_data_spacy = convert_conll_to_spacy(conll["test"])

### Step 3: Using spaCy declare a pre-trained model, train it on the conll data.

In [ ]:
def train_ner(train_data, n_iter=3, model="en_core_web_sm"):
    nlp = spacy.load(model)
    print("Loaded model")
    ner = nlp.get_pipe("ner")


    for _, annotations in train_data:
        for ent in annotations["entities"]:
            ner.add_label(ent[2])

    unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]


    optimizer = nlp.resume_training()

    with nlp.disable_pipes(*unaffected_pipes):
        for i in range(n_iter):
            random.shuffle(train_data)
            losses = {}

            for text, annotations in train_data:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)

                nlp.update([example], drop=0.3, losses=losses)

            print(f"Iteration {i+1}: Losses {losses}")

    return nlp

In [ ]:
train_data_spacy = train_data_spacy[:2000]
nlp_custom = train_ner(train_data_spacy, n_iter=3)

Loaded model
Iteration 1: Losses {'ner': 2956.4333416357535}
Iteration 2: Losses {'ner': 1524.285870418563}
Iteration 3: Losses {'ner': 1137.8897192841446}


### Step 4: Model Evaluation.

In [ ]:
def evaluate_model(nlp, test_data):
    y_true, y_pred = [], []

    for text, annotations in test_data:
        doc = nlp(text)
        true_tags = offsets_to_biluo_tags(doc, annotations["entities"])
        pred_tags = [token.ent_iob_ + ("-" + token.ent_type_ if token.ent_type_ else "")
                     for token in doc]
        y_true.extend(true_tags)
        y_pred.extend(pred_tags)

    print("NER Model Evaluation Report:")
    print(classification_report(y_true, y_pred))

In [ ]:
evaluate_model(nlp_custom, test_data_spacy)

NER Model Evaluation Report:


C:\AnacondaPython\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\AnacondaPython\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\AnacondaPython\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\AnacondaPython\lib\site-packages\sklearn\metrics\_classification.py:1344: Undefin

              precision    recall  f1-score   support

     B-B-LOC       0.00      0.20      0.00        10
    B-B-MISC       0.03      0.47      0.06        49
     B-B-ORG       0.00      0.00      0.00        20
     B-B-PER       0.00      0.12      0.00         8
     B-I-LOC       0.00      0.00      0.00         0
    B-I-MISC       0.00      0.33      0.01         3
     B-I-ORG       0.00      0.00      0.00         1
     B-I-PER       0.01      0.65      0.02        17
     I-B-LOC       0.00      0.00      0.00         9
    I-B-MISC       0.12      0.45      0.18        51
     I-B-ORG       0.00      0.00      0.00        19
     I-B-PER       0.00      0.00      0.00        10
    I-I-MISC       0.00      0.00      0.00         3
     I-I-ORG       0.00      0.00      0.00         1
     I-I-PER       0.00      0.00      0.00        19
     L-B-LOC       0.00      0.00      0.00        10
    L-B-MISC       0.00      0.00      0.00        49
     L-B-ORG       0.00    

C:\AnacondaPython\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\AnacondaPython\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Step 5: Mapping Entity to meaningful names

In [ ]:
entity_meanings = {
    "B-PER": "Person",
    "I-PER": "Person",
    "B-ORG": "Organization",
    "I-ORG": "Organization",
    "B-LOC": "Location",
    "I-LOC": "Location",
    "B-MISC": "Miscellaneous",
    "I-MISC": "Miscellaneous"
}

def test_finetuned_ner(nlp_finetuned, text):
    """Test the fine-tuned NER model and provide entity descriptions."""
    doc = nlp_finetuned(text)
    print("Entities found:\n")

    for ent in doc.ents:
        entity_label = ent.label_
        description = entity_meanings.get(entity_label, "Unknown")
        print(f"{ent.text} -> {entity_label} ({description})")

# Example test case
sample_text = "Google was founded by Larry Page and Sergey Brin in 1998 at Stanford University."
test_finetuned_ner(nlp_custom, sample_text)


Entities found:

Google -> B-ORG (Organization)
Larry -> B-PER (Person)
Page -> I-PER (Person)
Sergey -> B-PER (Person)
Brin -> I-PER (Person)
Stanford -> B-ORG (Organization)
University -> I-ORG (Organization)
